In [1]:
import re
import pandas as pd

mlb = pd.read_csv('mlb_team_payrolls.csv')
mlb["Year"] = mlb["Year"].astype(str)
team_names = {
    "NYY": "New York Yankees",
    "PHI": "Philadelphia Phillies",
    "LAD": "Los Angeles Dodgers",
    "NYM": "New York Mets",
    "CHC": "Chicago Cubs",
    "MIN": "Minnesota Twins",
    "SEA": "Seattle Mariners",
    "KC": "Kansas City Royals",
    "COL": "Colorado Rockies",
    "BOS": "Boston Red Sox",
    "CHW": "Chicago White Sox",
    "MIL": "Milwaukee Brewers",
    "BAL": "Baltimore Orioles",
    "WSH": "Washington Nationals",
    "CIN": "Cincinnati Reds",
    "PIT": "Pittsburgh Pirates",
    "CLE": "Cleveland Guardians",
    "LAA": "Los Angeles Angels",
    "TB": "Tampa Bay Rays",
    "DET": "Detroit Tigers",
    "OAK": "Oakland Athletics",
    "MIA": "Miami Marlins",
    "ARI": "Arizona Diamondbacks",
    "ATL": "Atlanta Braves",
    "HOU": "Houston Astros",
    "SD": "San Diego Padres",
    "SF": "San Francisco Giants",
    "STL": "St. Louis Cardinals",
    "TEX": "Texas Rangers",
    "TOR": "Toronto Blue Jays",
}


mlb["Team Full"] = mlb["Team"].map(team_names).fillna(mlb["Team"])
mlb[["wins", "losses"]] = mlb["Record"].str.split("-", expand=True)
mlb

,Team,Record,Avg Age,Total Payroll,Active,Injured,Retained,Buried,Year,Team Full,wins,losses
0,NYY,94-68,29.1,314741274.0,260398591.0,25630756.0,12213158.0,3565533.0,2024,New York Yankees,94,68
1,PHI,95-67,29.0,247291271.0,226203004.0,3031516.0,9697997.0,1699868.0,2024,Philadelphia Phillies,95,67
2,LAD,98-64,30.4,265870208.0,198627830.0,37949016.0,8698968.0,1846901.0,2024,Los Angeles Dodgers,98,64
3,NYM,89-73,30.4,317483315.0,189012437.0,37578740.0,7505378.0,3931503.0,2024,New York Mets,89,73
4,CHC,83-79,27.9,232067975.0,177805468.0,10207418.0,8538860.0,7604537.0,2024,Chicago Cubs,83,79
...,...,...,...,...,...,...,...,...,...,...,...,...
295,TB,80-82,27.1,73241552.0,52092948.0,8440963.0,12164597,543044,2015,Tampa Bay Rays,80,82
296,CLE,92-69,27.1,77404413.0,45520977.0,680720.0,19571148,1385666,2015,Cleveland Guardians,92,69
297,ARI,89-73,26.0,76781801.0,45142733.0,1026500.0,30465028,147540,2015,Arizona Diamondbacks,89,73
298,OAK,69-93,27.9,80491339.0,42311467.0,8772896.0,29168581,238395,2015,Oakland Athletics,69,93


In [2]:
# Load official standings for wins
standings = pd.read_csv("standings.csv")
# Rename the necessary columns
standings.rename(columns={
    "Season": "Year",
    "W": "Wins"
}, inplace=True)

standings["Year"] = standings["Year"].astype(str)
standings

,Rk,Tm,Wins,L,W-L%,R,RA,Rdiff,SOS,SRS,...,Home,Road,ExInn,1Run,vRHP,vLHP,≥.500,<.500,Year,Team
0,1.0,Los Angeles Angels of Anaheim,98,64,0.605,4.8,3.9,0.9,0.1,1.0,...,52-29,46-35,10-7,27-22,70-44,28-20,41-43,57-21,2014,LAA
1,2.0,Washington Nationals,96,66,0.593,4.2,3.4,0.8,-0.2,0.7,...,51-30,45-36,8-9,26-22,68-49,28-17,23-23,73-43,2014,WSH
2,3.0,Baltimore Orioles,96,66,0.593,4.4,3.7,0.7,0.1,0.8,...,50-31,46-35,14-6,32-23,74-47,22-19,51-40,45-26,2014,BAL
3,4.0,Los Angeles Dodgers,94,68,0.580,4.4,3.8,0.6,-0.2,0.4,...,45-36,49-32,6-12,25-20,68-52,26-16,26-33,68-35,2014,LAD
4,5.0,St. Louis Cardinals,90,72,0.556,3.8,3.7,0.1,-0.2,-0.1,...,51-30,39-42,7-6,32-23,69-54,21-18,39-33,51-39,2014,STL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,26.0,Oakland Athletics,69,93,0.426,4.0,4.7,-0.7,0.0,-0.8,...,38-43,31-50,5-10,28-26,49-74,20-19,33-65,36-28,2024,OAK
326,27.0,Los Angeles Angels,63,99,0.389,3.9,4.9,-1.0,0.0,-1.0,...,32-49,31-50,4-8,22-26,49-79,14-20,40-58,23-41,2024,LAA
327,28.0,Miami Marlins,62,100,0.383,3.9,5.2,-1.3,0.1,-1.1,...,30-51,32-49,10-9,23-20,51-55,11-45,42-61,20-39,2024,MIA
328,29.0,Colorado Rockies,61,101,0.377,4.2,5.7,-1.5,0.2,-1.3,...,37-44,24-57,8-4,26-21,46-69,15-32,42-63,19-38,2024,COL


In [3]:
# Merge payrolls with standings
mlb = mlb.merge(
    standings[["Year", "Team", "Wins"]],
    left_on=["Year", "Team"],
    right_on=["Year", "Team"],
    how="left"
)
# Keep only necessary columns
mlb = mlb[["Total Payroll", "Team Full", "Year", "Wins"]]
mlb

,Total Payroll,Team Full,Year,Wins
0,314741274.0,New York Yankees,2024,94
1,247291271.0,Philadelphia Phillies,2024,95
2,265870208.0,Los Angeles Dodgers,2024,98
3,317483315.0,New York Mets,2024,89
4,232067975.0,Chicago Cubs,2024,83
...,...,...,...,...
295,73241552.0,Tampa Bay Rays,2015,80
296,77404413.0,Cleveland Guardians,2015,81
297,76781801.0,Arizona Diamondbacks,2015,79
298,80491339.0,Oakland Athletics,2015,68


In [5]:
# Final payroll dataframe
mlb_final = mlb.rename(
    columns={"Year": "year", "Team Full": "team", "Total Payroll": "team_salary", "Wins": "wins"}
)
mlb_final = mlb_final[["year", "team", "team_salary", "wins"]]

postseason_df = pd.read_csv("mlb_postseason_2015-2024.csv")
postseason_df["Year"] = postseason_df["Year"].astype(str)

def parse_team_record(team_str):
    m = re.match(r"^(.*?)\s+\((\d+)-(\d+),\s*\w+\)$", team_str)
    return (
        (m.group(1).rstrip("*").strip(), int(m.group(2)), int(m.group(3)))
        if m
        else (None, None, None)
    )

postseason_df[["Winner_Team", "Winner_Wins", "Winner_Losses"]] = postseason_df[
    "Winner"
].apply(lambda x: pd.Series(parse_team_record(x)))
postseason_df[["Loser_Team", "Loser_Wins", "Loser_Losses"]] = postseason_df[
    "Loser"
].apply(lambda x: pd.Series(parse_team_record(x)))


In [6]:

winner_df = (
    postseason_df.groupby(["Year", "Winner_Team"])
    .agg(playoff_wins=("Wins", "sum"))
    .reset_index()
    .rename(columns={"Year": "year", "Winner_Team": "team"})
)
loser_df = (
    postseason_df.groupby(["Year", "Loser_Team"])
    .agg(playoff_wins=("Losses", "sum"))
    .reset_index()
    .rename(columns={"Year": "year", "Loser_Team": "team"})
)

playoff_wins_df = (
    pd.concat([winner_df, loser_df], ignore_index=True)
    .groupby(["year", "team"])
    .agg(playoff_wins=("playoff_wins", "sum"))
    .reset_index()
)

champions_df = postseason_df[postseason_df["Series"].str.contains("World Series")][
    ["Year", "Winner_Team"]
].rename(columns={"Year": "year", "Winner_Team": "team"})
champions_df["champion"] = True


In [7]:

mlb_final = mlb_final.merge(playoff_wins_df, on=["year", "team"], how="left")
mlb_final["playoff_wins"] = mlb_final["playoff_wins"].fillna(0).astype(int)
mlb_final = mlb_final.merge(champions_df, on=["year", "team"], how="left")
mlb_final["champion"] = mlb_final["champion"].fillna(False)

salary_quartiles = (
    mlb_final.groupby("year")["team_salary"]
    .quantile([0.25, 0.75])
    .unstack()
    .reset_index()
)
salary_quartiles.columns = ["year", "Q1", "Q3"]
mlb_final = mlb_final.merge(salary_quartiles, on="year", how="left")


/var/folders/1n/8nmzbrw53512w_qq871l3pw40000gn/T/ipykernel_66002/4000223723.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mlb_final["champion"] = mlb_final["champion"].fillna(False)


In [8]:

def assign_spend_tier(row):
    if row["team_salary"] < row["Q1"]:
        return "Low"
    elif row["team_salary"] > row["Q3"]:
        return "High"
    else:
        return "Medium"

mlb_final["spend_tier"] = mlb_final.apply(assign_spend_tier, axis=1)
mlb_final.drop(columns=["Q1", "Q3"], inplace=True)
mlb_final["sport"] = "MLB"

In [15]:
mlb_final.to_csv("mlb_wins.csv", index=False)